# Final Project 

For this week, my main goal was to set up my data and clearly define what my input and output would be so that I could connect my dataset to the active learning and embedding ideas described in the article and the GitHub code. My data consisted of a file containing images of different animal species, and the input data was the raw animal images, while the output data was the predicted species label that the already existing model from the paper would learn to assign to each image. I was not focusing on plotting this week, and instead, I was planning how to use the code from the published model to process my images and generate the learned embeddings that the classifier in the paper uses for species prediction. I have not run this part yet because I am still new to this code and trying to understand how the pipeline works. I was not trying to create a new model myself and was instead preparing to use the trained model and workflow described in the paper, which goes from an image to a crop to an embedding and then to a final label, although my project uses this at a smaller scale. The goal of this approach was to see whether I could use the paper’s method to classify multiple species in my own dataset, even with a limited number of labeled images. My hypothesis was that the model would perform better on animals that appeared more frequently in the dataset such as deer, lions or elephants, and might struggle with species that had fewer examples, such as wild boar monkeys or leopards, which was consistent with the patterns observed in the article, and I also expected the results to improve as more labeled examples were added.


For my project, the machine learning method that fits best with my data is the Random Forest Classifier from section 1.11.2 of the Scikit Learn ensemble methods. My data consists of animal images that will be converted into feature vectors using an embedding model, just like the approach described in the article and Random Forests work very well with this kind of high dimensional structured input. A Random Forest creates many decision trees that each learn slightly different patterns because of built-in randomness, which makes the final model more stable and less likely to overfit. This is especially helpful for my dataset because I expect some species to have more examples than others, and the ensemble approach helps the model stay balanced. Random Forests can also handle multi-class classification naturally, which is necessary because I am predicting multiple species. They are easy to train, interpret, and tune, which makes them a reliable and practical choice for my final project.


